<a href="https://colab.research.google.com/github/404S-retr0/Even_Dead_I_Am-_The_Hero-E.D.I.T.H-/blob/main/friday_lang_detect.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import string
import re
import codecs
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn import feature_extraction
from sklearn import linear_model 
from sklearn import pipeline 
from sklearn import metrics
from sklearn.model_selection import train_test_split
!wget https://raw.githubusercontent.com/404S-retr0/Even_Dead_I_Am-_The_Hero-E.D.I.T.H-/main/english_us_wordlist_top_100k.txt
!wget https://raw.githubusercontent.com/404S-retr0/Even_Dead_I_Am-_The_Hero-E.D.I.T.H-/main/czech_wordlist_top_100k_2016_02_11.txt
!wget https://raw.githubusercontent.com/404S-retr0/Even_Dead_I_Am-_The_Hero-E.D.I.T.H-/main/croatian_wordlist_top_100k.txt
!wget https://raw.githubusercontent.com/404S-retr0/Even_Dead_I_Am-_The_Hero-E.D.I.T.H-/main/dutch_wordlist_top_100k_2012_05.txt
!wget https://raw.githubusercontent.com/404S-retr0/Even_Dead_I_Am-_The_Hero-E.D.I.T.H-/main/danish_wordlist_top_100k_2011_12.txt
eng_df = pd.read_csv("english_us_wordlist_top_100k.txt","utf-8",header=None,names=["English"]) 
eng_df
cro_df = pd.read_csv("croatian_wordlist_top_100k.txt","utf-8",header=None,names=["Croatian"]) 
cro_df
cze_df = pd.read_csv("czech_wordlist_top_100k_2016_02_11.txt","utf-8",header=None,names=["Czech"]) 
cze_df
dan_df = pd.read_csv("danish_wordlist_top_100k_2011_12.txt","utf-8",header=None,names=["Danish"]) 
dan_df
dut_df = pd.read_csv("dutch_wordlist_top_100k_2012_05.txt","utf-8",header=None,names=["Dutch"]) 
dut_df

In [ ]:
for char in string.punctuation:
  print(char,end=" ")
translate_table = dict ((ord(char),None) for char in string.punctuation)

! " # $ % & ' ( ) * + , - . / : ; < = > ? @ [ \ ] ^ _ ` { | } ~ 

In [ ]:
data_eng = []
lang_eng = []
for i,line in eng_df.iterrows():
  line = line['English']
  if len(line) !=0:
    line = line.lower()
    line = re.sub(r"\d+","",line)
    line = line.translate(translate_table)
    data_eng.append(line)
    lang_eng.append("English")

In [ ]:
data_cro = []
lang_cro = []
for i,line in cro_df.iterrows():
  line = line['Croatian']
  if len(line) !=0:
    line = line.lower()
    line = re.sub(r"\d+","",line)
    line = line.translate(translate_table)
    data_cro.append(line)
    lang_cro.append("Croatian")

In [ ]:
data_cze = []
lang_cze = []
for i,line in cze_df.iterrows():
  line = line['Czech']
  if len(line) !=0:
    line = line.lower()
    line = re.sub(r"\d+","",line)
    line = line.translate(translate_table)
    data_cze.append(line)
    lang_cze.append("Czech")

In [ ]:
data_dan = []
lang_dan = []
for i,line in dan_df.iterrows():
  line = line['Danish']
  if len(line) !=0:
    line = line.lower()
    line = re.sub(r"\d+","",line)
    line = line.translate(translate_table)
    data_dan.append(line)
    lang_dan.append("Danish")

In [ ]:
data_dut = []
lang_dut = []
for i,line in dut_df.iterrows():
  line = line['Dutch']
  if len(line) !=0:
    line = line.lower()
    line = re.sub(r"\d+","",line)
    line = line.translate(translate_table)
    data_dut.append(line)
    lang_dut.append("Dutch")

In [ ]:
df=pd.DataFrame({"Text":data_eng+data_cro+data_cze+data_dan+data_dut,
                 "Language":lang_eng+lang_cro+lang_cze+lang_dan+lang_dut})
print(df.shape)
df

In [ ]:
x,y=df.iloc[:,0],df.iloc[:,1]
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=0)

print(x_train.shape)
print(x_test.shape)
print(y_train.shape)
print(y_test.shape)

In [ ]:
vectorizer = feature_extraction.text.TfidfVectorizer(ngram_range=(1,2),analyzer='char')
pipe_lr_r13 = pipeline.Pipeline([
                                 ('vectorizer',vectorizer),
                                 ('clf',linear_model.LogisticRegression())
])

In [ ]:
pipe_lr_r13.fit(x_train,y_train)

In [ ]:
y_predicted = pipe_lr_r13.predict(x_test)

In [ ]:
acc = (metrics.accuracy_score(y_test,y_predicted))*100
print(acc,'%')

In [ ]:
matrix = metrics.confusion_matrix(y_test,y_predicted)
print('confusion matrix : \n',matrix)

In [ ]:
import pickle
lrFile = open('LRModel.pckl','wb')
pickle.dump(pipe_lr_r13,lrFile)
lrFile.close()

In [ ]:
def lang_detect(text):
  import numpy as np
  import string
  import re
  import pickle
  translate_table = dict ((ord(char),None) for char in string.punctuation)

  global lrLangDetectModel
  lrLangDetectFile = open('LRModel.pckl','rb')
  lrLangDetectModel = pickle.load(lrLangDetectFile)
  lrLangDetectFile.close();

  text = " ".join(text.split())
  text = text.lower()
  text = re.sub(r"\d","",text)
  text = text.translate(translate_table)
  pred = lrLangDetectModel.predict([text])
  prob = lrLangDetectModel.predict_proba([text])
  return pred[0]

In [ ]:
lang_detect("hello i just built my own language detection model")

In [ ]:
lang_detect("vi")